In [1]:
# import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import pathlib
import itertools
import os
import pickle

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = pd.DataFrame(columns=['path','date', 'year','month'])
df.path = [str(x) for x in pathlib.Path('data/').glob('**/*.pkl')]
df.month = df.path.apply(lambda x: x[5:11])
df.year = df.path.apply(lambda x: x[5:9])
df.date = df.path.apply(lambda x: x[5:13])
# df.source = df.path.apply(lambda x: x.split('/')[0])
# df.filename = df.path.apply(lambda x: x.split('/')[1])
# df['date'] = df.path.apply(lambda x: re.search(r'_(\d\d\d\d\d\d)',x)[1]).values
# df['year'] = df.date.apply(lambda x: x[0:4]).values.astype(int)
# df['output'] = '423/'+ df['source']+'_' +df['date']+'.csv'
df = df[df.date.astype(int)<20130401].sort_values('path').reset_index(drop=True)
df

,path,date,year,month
0,data/20070101.pkl,20070101,2007,200701
1,data/20070102.pkl,20070102,2007,200701
2,data/20070103.pkl,20070103,2007,200701
3,data/20070104.pkl,20070104,2007,200701
4,data/20070105.pkl,20070105,2007,200701
...,...,...,...,...
2276,data/20130327.pkl,20130327,2013,201303
2277,data/20130328.pkl,20130328,2013,201303
2278,data/20130329.pkl,20130329,2013,201303
2279,data/20130330.pkl,20130330,2013,201303


In [3]:
def get_rows(path):
    with open(path, 'rb') as f:
        data = pickle.load(f, encoding = 'latin1')
        return data

In [4]:
pandarallel.initialize(nb_workers=32, progress_bar=False)    
df1 = df.copy()
df1['data'] = df1['path'].parallel_apply(get_rows) 
df1

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,path,date,year,month,data
0,data/20070101.pkl,20070101,2007,200701,"[{'href': 'http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102', 'ts': '20070101Â 11:47 PM EST', 'title': 'UPDATE 1-TSMC plans five new advanced wafer plants -paper'}, {'href': 'http://www.reuters.com/article/musicNews/idUSN3126672520070102', 'ts': '20070101Â 11:40 PM EST', 'title': 'U2 roadie gets a moment in the spotlight'}, {'href': 'http://www.reuters.com/article/globalNews/idUSHAN8311820070102', 'ts': '20070101Â 11:12 PM EST', 'title': 'Vietnamese family tested negative for bird flu'}, {'href': 'http://www.reuters.com/article/musicNews/idUSN2927338620070102', 'ts': '2..."
1,data/20070102.pkl,20070102,2007,200701,"[{'href': 'http://www.reuters.com/article/sportsNews/idUSSP13581520070103', 'ts': '20070102Â 11:53 PM EST', 'title': 'Appleby right at home in bid for fourth Kapalua win'}, {'href': 'http://www.reuters.com/article/worldNews/idUSL0188670820070103', 'ts': '20070102Â 11:38 PM EST', 'title': 'FACTBOX-Indonesian air disasters'}, {'href': 'http://www.reuters.com/article/companyNewsAndPR/idUSTP14067520070103', 'ts': '20070102Â 11:16 PM EST', 'title': 'Global chip sales to rise 11 pct in 2007 - iSuppli'}, {'href': 'http://www.reuters.com/article/ousiv/idUSL0290207520070103', 'ts': '20070102Â 11:14..."
2,data/20070103.pkl,20070103,2007,200701,"[{'href': 'http://www.reuters.com/article/companyNewsAndPR/idUST27294020070104', 'ts': '20070103Â 11:57 PM EST', 'title': 'TREASURIES-Sit tight in Asia, await data'}, {'href': 'http://www.reuters.com/article/sportsNews/idUSSP24751620070104', 'ts': '20070103Â 11:53 PM EST', 'title': 'Henin to miss Open warm-up for ""personal reasons""'}, {'href': 'http://www.reuters.com/article/companyNewsAndPR/idUSSHA17012320070104', 'ts': '20070103Â 11:48 PM EST', 'title': 'UPDATE 1-Hyundai Motor China JV sees slow 2007 sales rise'}, {'href': 'http://www.reuters.com/article/worldNews/idUSL0428761520070104',..."
3,data/20070104.pkl,20070104,2007,200701,"[{'href': 'http://www.reuters.com/article/sportsNews/idUSB86860320070105', 'ts': '20070104Â 11:51 PM EST', 'title': 'Leafs score nine straight goals to crush Bruins'}, {'href': 'http://www.reuters.com/article/ousiv/idUSL0552510920070105', 'ts': '20070104Â 11:29 PM EST', 'title': 'Shareholder warns Vodafone on India venture, paper says'}, {'href': 'http://www.reuters.com/article/businessNews/idUSSYU00145320070105', 'ts': '20070104Â 11:28 PM EST', 'title': 'QBE to buy Winterthur for $1.2 bln'}, {'href': 'http://www.reuters.com/article/companyNewsAndPR/idUSSYD18777420070105', 'ts': '20070104Â..."
4,data/20070105.pkl,20070105,2007,200701,"[{'href': 'http://www.reuters.com/article/sportsNews/idUSN0530289020070106', 'ts': '20070105Â 11:47 PM EST', 'title': 'Singh moves one ahead at wet and windy Kapalua'}, {'href': 'http://www.reuters.com/article/healthNews/idUSN0530828220070106', 'ts': '20070105Â 11:09 PM EST', 'title': 'Some doctors trim use of anemia drugs: surveys'}, {'href': 'http://www.reuters.com/article/sportsNews/idUSSP17595420070106', 'ts': '20070105Â 11:04 PM EST', 'title': 'Denver's Iverson fined $25,000 for referee remarks'}, {'href': 'http://www.reuters.com/article/companyNewsAndPR/idUSL0672063220070106', 'ts': ..."
...,...,...,...,...,...
2276,data/20130327.pkl,20130327,2013,201303,"[{'href': 'http://www.reuters.com/article/2013/03/28/justice-breuer-idUSL2N0CJ1OC20130328', 'ts': '20130327Â 11:59PM UTC', 'title': 'Ex-DOJ criminal chief Breuer returns to Covington & Burling'}, {'href': 'http://www.reuters.com/article/2013/03/28/eletrobras-results-idUSL2N0CK03X20130328', 'ts': '20130327Â 11:58PM UTC', 'title': 'UPDATE 1-Brazil's Eletrobras posts $3.4 billion loss for 2012'}, {'href': 'http://www.reuters.com/article/2013/03/28/shell-alaska-idUSL2N0CK02G20130328', 'ts': '20130327Â 11:56PM UTC', 'title': 'Shell faces new probe over Alaska drill program'}, {'href': 'http://w..."
2277,data/20130328.pkl,20130328,2013,201303,"[{'href': 'http://www.reuters.com/article

In [5]:
df1= df1.explode('data')
df1 = df1[~df1.data.isnull()].reset_index(drop=True)
df1

,path,date,year,month,data
0,data/20070101.pkl,20070101,2007,200701,"{'href': 'http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102', 'ts': '20070101Â 11:47 PM EST', 'title': 'UPDATE 1-TSMC plans five new advanced wafer plants -paper'}"
1,data/20070101.pkl,20070101,2007,200701,"{'href': 'http://www.reuters.com/article/musicNews/idUSN3126672520070102', 'ts': '20070101Â 11:40 PM EST', 'title': 'U2 roadie gets a moment in the spotlight'}"
2,data/20070101.pkl,20070101,2007,200701,"{'href': 'http://www.reuters.com/article/globalNews/idUSHAN8311820070102', 'ts': '20070101Â 11:12 PM EST', 'title': 'Vietnamese family tested negative for bird flu'}"
3,data/20070101.pkl,20070101,2007,200701,"{'href': 'http://www.reuters.com/article/musicNews/idUSN2927338620070102', 'ts': '20070101Â 10:58 PM EST', 'title': 'Chicago jazz mecca closes doors, perhaps forever'}"
4,data/20070101.pkl,20070101,2007,200701,"{'href': 'http://www.reuters.com/article/entertainmentNews/idUSN2820662920070102', 'ts': '20070101Â 10:56 PM EST', 'title': '""Factory Girl"" affords more fame to Warhol muse'}"
...,...,...,...,...,...
5918463,data/20130331.pkl,20130331,2013,201303,"{'href': 'http://www.reuters.com/news/video/videoStory?storyID=58441026', 'ts': '20130331Â 6:28AM UTC', 'title': 'Airlines Qantas and Emirates strike alliance'}"
5918464,data/20130331.pkl,20130331,2013,201303,"{'href': 'http://www.reuters.com/news/video/videoStory?storyID=58439639', 'ts': '20130331Â 4:36AM UTC', 'title': 'Christians celebrate Jesus' resurrection in Jerusalem'}"
5918465,data/20130331.pkl,20130331,2013,201303,"{'href': 'http://www.reuters.com/news/video/videoStory?storyID=58439043', 'ts': '20130331Â 3:55AM UTC', 'title': 'At least 28 dead in mine blast'}"
5918466,data/20130331.pkl,20130331,2013,201303,"{'href': 'http://www.reuters.com/news/video/videoStory?storyID=58438708', 'ts': '20130331Â 3:49AM UTC', 'title': 'Weapons handed over to Mali troops'}"


In [6]:
df2 = pd.json_normalize(df1.data).reset_index(drop=True)
df2

,href,ts,title
0,http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102,20070101Â 11:47 PM EST,UPDATE 1-TSMC plans five new advanced wafer plants -paper
1,http://www.reuters.com/article/musicNews/idUSN3126672520070102,20070101Â 11:40 PM EST,U2 roadie gets a moment in the spotlight
2,http://www.reuters.com/article/globalNews/idUSHAN8311820070102,20070101Â 11:12 PM EST,Vietnamese family tested negative for bird flu
3,http://www.reuters.com/article/musicNews/idUSN2927338620070102,20070101Â 10:58 PM EST,"Chicago jazz mecca closes doors, perhaps forever"
4,http://www.reuters.com/article/entertainmentNews/idUSN2820662920070102,20070101Â 10:56 PM EST,"""Factory Girl"" affords more fame to Warhol muse"
...,...,...,...
5918463,http://www.reuters.com/news/video/videoStory?storyID=58441026,20130331Â 6:28AM UTC,Airlines Qantas and Emirates strike alliance
5918464,http://www.reuters.com/news/video/videoStory?storyID=58439639,20130331Â 4:36AM UTC,Christians celebrate Jesus' resurrection in Jerusalem
5918465,http://www.reuters.com/news/video/videoStory?storyID=58439043,20130331Â 3:55AM UTC,At least 28 dead in mine blast
5918466,http://www.reuters.com/news/video/videoStory?storyID=58438708,20130331Â 3:49AM UTC,Weapons handed over to Mali troops


In [7]:
df = pd.concat([df1,df2], axis = 1)
df = df.drop(columns =['path','data','ts'])
df

,date,year,month,href,title
0,20070101,2007,200701,http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102,UPDATE 1-TSMC plans five new advanced wafer plants -paper
1,20070101,2007,200701,http://www.reuters.com/article/musicNews/idUSN3126672520070102,U2 roadie gets a moment in the spotlight
2,20070101,2007,200701,http://www.reuters.com/article/globalNews/idUSHAN8311820070102,Vietnamese family tested negative for bird flu
3,20070101,2007,200701,http://www.reuters.com/article/musicNews/idUSN2927338620070102,"Chicago jazz mecca closes doors, perhaps forever"
4,20070101,2007,200701,http://www.reuters.com/article/entertainmentNews/idUSN2820662920070102,"""Factory Girl"" affords more fame to Warhol muse"
...,...,...,...,...,...
5918463,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58441026,Airlines Qantas and Emirates strike alliance
5918464,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58439639,Christians celebrate Jesus' resurrection in Jerusalem
5918465,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58439043,At least 28 dead in mine blast
5918466,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58438708,Weapons handed over to Mali troops


In [8]:
del df1, df2

In [10]:
df['links']= df.href.apply(lambda x: re.sub('/\d\d\d\d/\d+/\d+/','/', x))
df['links']= df.links.apply(lambda x: re.sub('https?://www.reuters.com/\w+/','', x))
df['links']= df.links.apply(lambda x: re.sub('\d\d\d\d/\d+/\d+/','/', x))
df['urls'] = df['links'].apply(lambda x: x.split('/'))
df

,date,year,month,href,title,links,urls
0,20070101,2007,200701,http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102,UPDATE 1-TSMC plans five new advanced wafer plants -paper,companyNewsAndPR/idUSTP13157220070102,"[companyNewsAndPR, idUSTP13157220070102]"
1,20070101,2007,200701,http://www.reuters.com/article/musicNews/idUSN3126672520070102,U2 roadie gets a moment in the spotlight,musicNews/idUSN3126672520070102,"[musicNews, idUSN3126672520070102]"
2,20070101,2007,200701,http://www.reuters.com/article/globalNews/idUSHAN8311820070102,Vietnamese family tested negative for bird flu,globalNews/idUSHAN8311820070102,"[globalNews, idUSHAN8311820070102]"
3,20070101,2007,200701,http://www.reuters.com/article/musicNews/idUSN2927338620070102,"Chicago jazz mecca closes doors, perhaps forever",musicNews/idUSN2927338620070102,"[musicNews, idUSN2927338620070102]"
4,20070101,2007,200701,http://www.reuters.com/article/entertainmentNews/idUSN2820662920070102,"""Factory Girl"" affords more fame to Warhol muse",entertainmentNews/idUSN2820662920070102,"[entertainmentNews, idUSN2820662920070102]"
...,...,...,...,...,...,...,...
5918463,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58441026,Airlines Qantas and Emirates strike alliance,video/videoStory?storyID=58441026,"[video, videoStory?storyID=58441026]"
5918464,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58439639,Christians celebrate Jesus' resurrection in Jerusalem,video/videoStory?storyID=58439639,"[video, videoStory?storyID=58439639]"
5918465,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58439043,At least 28 dead in mine blast,video/videoStory?storyID=58439043,"[video, videoStory?storyID=58439043]"
5918466,20130331,2013,201303,http://www.reuters.com/news/video/videoStory?storyID=58438708,Weapons handed over to Mali troops,video/videoStory?storyID=58438708,"[video, videoStory?storyID=58438708]"


In [12]:
df = df[df.urls.str.len()<3].reset_index(drop=True)
df.urls.str.len().value_counts()

2    2981128
1    2937224
Name: urls, dtype: int64

In [22]:
num_chunks = 100

df_save = np.array_split(df.sample(frac=1), num_chunks)
save_chunks = lambda x: df_save[x].to_csv('csv/%03d.csv'%x, index=False)
pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 
del df

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [18]:
def clean_chunks(x):
    filename = 'csv/%03d.csv'%x
    df = pd.read_csv(filename, header=0)
    
#     df = df[~df.links.str.contains('rbssBanks|money|health|living|autos|further|Compan|Energy|banks|restaurant|Food|popular|Wealth|Life|Work|Enterprise|Fund|Trading|Trade', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('retail|Entrepreneur|Transportation|Jobs|College|Metals|Science|Savings|goods|materials', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('power.?lunch|STYLE|TECH|NATURE|SHOWBIZ|WEATHER', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('inPlayBriefing|companyNews|/video/|bondsNews|hotStocksNews|\w*banking\w*|\w*stock\w*|oil\w*gas', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('video|mergers|Telecom|rbssEnergy|peopleNews|\w*film\w*|\w*music\w*|\w*environment\w*|bondsNews|arts\w*|\w*Market\w*|\w*Financ\w*|\w*econom\w*|\w*culture\w*|\w*business\w*|\w*fashion\w*|sport\w*|lifestyle\w*|studentnews|travel\w*|entertainment\w*|obituaries', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('Mkt\w+|Consumer\w*|health\w*|industr\w*|product\w*|tech\w*', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('huginPressRelease|boxOffice|television|ousivMolt|innovation', flags=re.IGNORECASE)]
#     df = df[~df.links.str.contains('etfNews|olympicsNews|Dollar|ousiv|RealEstate|Currency|telcommunication|privateCapital|earning|CMPSRV|Manufactur|Investment|rbss|Service', flags=re.IGNORECASE)]
    
    df = df[~df.links.str.contains('pressReleasesMolt|oddlyEnoughNews|OIL|bankrupt|privateEquity', flags=re.IGNORECASE)]
    
       
    
    df.to_csv(filename, index=False)

num_chunks=100    
pandarallel.initialize(nb_workers=32, progress_bar=False)       
_ = pd.Series(range(num_chunks)).parallel_apply(clean_chunks) 

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [19]:
df= pd.read_csv('csv/002.csv',header=0)
df

,date,year,month,href,title,links,urls
0,20120508,2012,201205,http://www.reuters.com/article/2012/05/08/pershingsquare-canadianpacificrailway-idUSL1E8G86IA20120508,Pershing's Ackman prefers vote on CP Rail future,pershingsquare-canadianpacificrailway-idUSL1E8G86IA20120508,['pershingsquare-canadianpacificrailway-idUSL1E8G86IA20120508']
1,20120903,2012,201209,http://www.reuters.com/article/2012/09/03/toyota-china-sales-idUSL4E8K31FV20120903,UPDATE 1-Toyota's China sales in August fall 15 percent,toyota-china-sales-idUSL4E8K31FV20120903,['toyota-china-sales-idUSL4E8K31FV20120903']
2,20101103,2010,201011,http://www.reuters.com/article/2010/11/03/idUS191701+03-Nov-2010+PRN20101103,Toyota Reports October Sales,idUS191701+03-Nov-2010+PRN20101103,['idUS191701+03-Nov-2010+PRN20101103']
3,20120112,2012,201201,http://www.reuters.com/article/2012/01/12/idUS49704+12-Jan-2012+PRN20120112,Following Is a Test Release,idUS49704+12-Jan-2012+PRN20120112,['idUS49704+12-Jan-2012+PRN20120112']
4,20080314,2008,200803,http://www.reuters.com/article/pressRelease/idUS107820+14-Mar-2008+PRN20080314,IMAX Corporation reports fourth quarter and fiscal 2007 financial results,pressRelease/idUS107820+14-Mar-2008+PRN20080314,"['pressRelease', 'idUS107820+14-Mar-2008+PRN20080314']"
...,...,...,...,...,...,...,...
45556,20081002,2008,200810,http://www.reuters.com/article/pressRelease/idUS151997+02-Oct-2008+PRN20081002,Sandra Lee's Highly Anticipated Annual Halloween Episode Resurrects Queens for 'Semi-Homemade...,pressRelease/idUS151997+02-Oct-2008+PRN20081002,"['pressRelease', 'idUS151997+02-Oct-2008+PRN20081002']"
45557,20080814,2008,200808,http://www.reuters.com/article/pressRelease/idUS210847+14-Aug-2008+BW20080814,Sequa Corporation Invitation for Investors to the Second Quarter FY 2008 Earnings...,pressRelease/idUS210847+14-Aug-2008+BW20080814,"['pressRelease', 'idUS210847+14-Aug-2008+BW20080814']"
45558,20120206,2012,201202,http://www.reuters.com/article/2012/02/06/idUS154527+06-Feb-2012+PRN20120206,Overhill Farms Announces First Quarter 2012 Earnings Conference Call,idUS154527+06-Feb-2012+PRN20120206,['idUS154527+06-Feb-2012+PRN20120206']
45559,20110125,2011,201101,http://www.reuters.com/article/2011/01/25/idUS161843+25-Jan-2011+PRN20110125,"Zacks Analyst Blog Highlights: Wendy's/Arby's Group, UBS, Yum! Brands, Cosi and Panera Bread",idUS161843+25-Jan-2011+PRN20110125,['idUS161843+25-Jan-2011+PRN20110125']


In [20]:
firstworld = df.links.apply(lambda x: x.split('/')[0])

In [21]:
firstworld.value_counts()[0:60]

pressRelease                                     15321
latestCrisis                                       985
topNews                                            241
worldNews                                          239
newsMaps                                           220
homepageCrisis                                     166
politicsNews                                       139
newsOne                                            134
domesticNews                                       131
reutersEdge                                         89
governmentFilingsNews                               81
mediaNews                                           65
managementIssues                                    52
BROKER                                              48
featuredCrisis                                      41
ELECTU                                              36
internetNews                                        36
CARMFG                                              33
GCA-Credit

In [17]:
firstworld.value_counts()[60:120]

BLDFIX                                             9
americasPrivateEquityNews                          8
euIpoNews                                          8
gc07                                               8
americasDealsNews                                  8
internal_ReutersNewsRoom_ExclusivesAndWins_MOLT    8
AGRLIV                                             8
asiaRegulatoryNews                                 7
internal_ReutersNewsRoom_BehindTheScenes_MOLT      7
gc03                                               7
FSCONS                                             7
GCA-Agflation                                      6
ADVERT                                             6
gc04                                               6
GCA-Iran                                           6
RAILRD                                             6
winterOlympicAlpine                                5
europeCrisis                                       5
GCA-CorporateCrime                            

In [22]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)

print('relevant_rules:')
print(relevant_rules)

with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

print('\nirelevant_rules:')
print(irelevant_rules)

relevant_rules:
re.compile('\\b\\w+-\\w+ relations|\\babduct|\\babuse|\\bactivist|\\bacts\\b|\\badministration|\\bagainst women|\\bagencies|\\ballegati|\\bambassador|\\banarch|\\barmed|\\barmy|\\barrest|\\bartillery|\\bassassin, re.IGNORECASE)

irelevant_rules:
re.compile('\\b/\\bstyles?\\b/|\\b/books?/|\\b/cars?/|\\b/earth/|\\b/food/|\\b/future/|\\b/health/|\\b/life/|\\b/living/|\\b/members?/|\\b/money/|\\b/packages?/|\\b/pictures?/|\\b/popular/|\\b/radio/|\\b/videos?, re.IGNORECASE)


In [26]:
def further_clean_chunks(x):
    filename = 'csv/%03d.csv'%x
    sizes=[]
    df = pd.read_csv(filename, header=0)
    sizes.append(df.shape[0])

    remove_index = df[df.links.str.contains(irelevant_rules) | df.title.str.contains(irelevant_rules)].index
    df = df[~df.index.isin(remove_index)]
    sizes.append(df.shape[0])
    
    new_filename = 'step1/%03d.csv'%x
    df.to_csv(new_filename, index=False)
    print(x, sizes)

num_chunks=100    
pandarallel.initialize(nb_workers=32, progress_bar=False)       
_ = pd.Series(range(num_chunks)).parallel_apply(further_clean_chunks) 

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
19 [45566, 29772]
16 [45624, 29774]
55 [45544, 29798]
46 [45738, 29984]
37 [45642, 29847]
85 [45681, 30000]
28 [45532, 29838]
73 [45565, 29688]
94 [45480, 29621]
22 [45419, 29665]
31 [45599, 29655]8
 [45539, 29816]
52 0[45474, 29765] 
[45542, 29477]
4912  [45606, 29701][45562, 29595]

61 [45540, 29731]
25 [45666, 29803]
40 [45613, 29800]
67 [45622, 29805]
97 [45458, 29493]479 
 [45694, 29858]91
[45468, 29700]
 [45524, 29768]
88 [45581, 29628]
76 [45601, 29714]
70 [45620, 29645]
64 [45588, 29839]
34 [45550, 29824]
82 [45440, 29864]
43 [45564, 30015]
58 [45661, 30096]
20 [45624, 29759]
17 [45322, 29585]
56 [45667, 29783]
38 [45530, 29770]
77 [45563, 29654]
47 [45669, 29999]
9 [45544, 29884]
86 [45780, 29885]
65 [45436, 29705]
13 [45599, 29853]
35 [45596, 29920]
29 [45568, 29908]
74 [45606, 29919]
98 [45599, 29866]
26 [45644, 29894]
59 [45542, 2985

In [2]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}
   
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
# config.request_timeout = 10

def extract_story(url):
    try:
        article = Article(url,config= config)
        time.sleep(1)
        article.download()
        article.parse()
        return {'url': url, 'text': article.text}
    except:
        try:
            urlid = url.split('/')[-1].split('-')[-1]
            if urlid.startswith('id'):
                new_page = 'https://www.reuters.com/article/instant-article/%s'%urlid
                article = Article(new_page,config= config)
                time.sleep(1)
                article.download()
                article.parse()
                return {'url': new_page, 'text': article.text}
            else:
                print('No instant %s'%url)
        except:
            print('error %s'%url)
        return

In [2]:
# import logging
from newspaper import Article, Config


user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10



def get_epoch_time(time_obj):
    if time_obj:
        return time_obj.timestamp()

    return None

def crawl_link_article(url):
    if 'theguardian' in url:
        time.sleep(4)
    if 'xinhuanet' in url:
        time.sleep(6)
    else:
        time.sleep(2)
    result_json = None

    try:
        if 'http' not in url:
            if url[0] == '/':
                url = url[1:]
            try:
#                 article = Article('http://' + url)
                article = Article('http://' + url,config= config)
                article.download()
                time.sleep(2)
                article.parse()
                flag = True
            except:
#                 print(url)
#                 logging.exception("Exception in getting data from url {}".format(url))
                flag = False
                pass
            if flag == False:
                try:
#                     article = Article('https://' + url)
                    article = Article('https://' + url,config= config)
                    article.download()
                    time.sleep(2)
                    article.parse()
                    flag = True
                except:
#                     logging.exception("Exception in getting data from url {}".format(url))
#                     print(url)
                    flag = False
                    pass
            if flag == False:
                return None
        else:
            try:
#                 article = Article(url)
                article = Article(url,config= config)
                article.download()
                time.sleep(2)
                article.parse()
            except:
                return None

        if not article.is_parsed:
            return None

        visible_text = article.text
        result_json = {'url': url, 'text': visible_text}
    except:
        pass

    return result_json

def crawl_news_article(url):
    news_article = crawl_link_article(url)

    # If the news article could not be fetched from original website, fetch from archieve if it exists.
    if news_article is None:
#         print('Failed\t%s'%url)
        archieve_url = get_website_url_from_arhieve(url)
        if archieve_url is None:
            print('None\t%s'%url)
        else:
            news_article = crawl_link_article(archieve_url)
            
            if news_article is None:
                print('404\t%s'%archieve_url)

    return news_article

def get_website_url_from_arhieve(url):
    """ Get the url from http://web.archive.org/ for the passed url if exists."""
    archieve_results = get_web_archieve_results(url)
    if archieve_results:
        modified_url = "https://web.archive.org/web/{}/{}".format(archieve_results[0][1], archieve_results[0][2])
        return modified_url
    else:
        return None
    
def get_web_archieve_results(search_url):
    try:
        archieve_url = "http://web.archive.org/cdx/search/cdx?url={}&output=json".format(search_url)

        response = requests.get(archieve_url)
        response_json = json.loads(response.content)

        response_json = response_json[1:]
        return response_json

    except:
        return None

In [9]:
pandarallel.initialize(nb_workers=32)
batch_size = 32*50

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [10]:
i = resume = 2240
while i< df1.shape[0]:
    print(i)
    df1.loc[i: i+ batch_size-1, 'articles']= df1.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_story).values
    df_temp = df1.loc[i: i+ batch_size-1]
    print('\tsuccess: %s/%s'%(df_temp[~df_temp.articles.isnull()].shape[0], df_temp.shape[0]))
    i += batch_size
    if (i-resume)% (batch_size * 10) ==0:
        df1.to_csv('step2/%03d.csv'%idx, index= False)
df1.to_csv('step2/%03d.csv'%idx, index= False)

2240
error http://www.reuters.com/article/2011/06/27/pictures-report-idUSRTR2O5KO
	success: 1599/1600
3840
	success: 1600/1600
5440
	success: 1600/1600
7040
error http://www.reuters.com/article/2010/10/29/idUS123792777920101029
	success: 237/238


In [13]:
pandarallel.initialize(nb_workers=32, progress_bar=False)
batch_size = 32*50

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [14]:
for idx in range(75, 90):
    print('\n ------------------- %s --------------------\n'%idx)
    df1 = pd.read_csv('step1/%03d.csv'%idx, header=0)[['date','href','title']]
    df1 = df1[~df1.href.str.contains('\+')].reset_index(drop=True)
    i=0
    while i< df1.shape[0]:
        print(i)
        df1.loc[i: i+ batch_size-1, 'articles']= df1.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_story).values
        df_temp = df1.loc[i: i+ batch_size-1]
        print('\tsuccess: %s/%s'%(df_temp[~df_temp.articles.isnull()].shape[0], df_temp.shape[0]))
        i += batch_size
        if i% (batch_size * 10) ==0:
            df1.to_csv('step2/%03d.csv'%idx, index= False)
    df1.to_csv('step2/%03d.csv'%idx, index= False)


 ------------------- 75 --------------------

0
error http://www.reuters.com/article/2011/01/28/idUS120446450320110128
	success: 1599/1600
1600
error http://www.reuters.com/article/2010/10/26/idUS166803456220101026
error http://www.reuters.com/article/2012/07/09/pictures-report-idUSRTR3395H
error http://www.reuters.com/article/2012/09/12/us-3sbio-offer-idUSBRE88B0W520120912
error http://www.reuters.com/article/mnGreenBuildings/idUS43537322520091006
error http://www.reuters.com/article/2012/09/25/pictures-report-idUSRTR38E3B
	success: 1595/1600
3200
error http://www.reuters.com/article/2011/03/08/idUS331740220720110308
	success: 1599/1600
4800
error http://www.reuters.com/article/2012/03/29/pictures-report-idUSRTR302P3
error http://www.reuters.com/article/2012/07/30/pictures-report-idUSRTR35N9B
error http://www.reuters.com/article/2010/12/28/idUS139476662920101228
error http://www.reuters.com/article/2012/04/25/idUS78170854420120425
error http://www.reuters.com/article/2011/09/05/pictu

In [15]:
for idx in [38,39]:
    print('\n ------------------- %s --------------------\n'%idx)
    df1 = pd.read_csv('step1/%03d.csv'%idx, header=0)[['date','href','title']]
    df1 = df1[~df1.href.str.contains('\+')].reset_index(drop=True)
    i=0
    while i< df1.shape[0]:
        print(i)
        df1.loc[i: i+ batch_size-1, 'articles']= df1.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_story).values
        df_temp = df1.loc[i: i+ batch_size-1]
        print('\tsuccess: %s/%s'%(df_temp[~df_temp.articles.isnull()].shape[0], df_temp.shape[0]))
        i += batch_size
        if i% (batch_size * 10) ==0:
            df1.to_csv('step2/%03d.csv'%idx, index= False)
    df1.to_csv('step2/%03d.csv'%idx, index= False)


 ------------------- 38 --------------------

0
error http://www.reuters.com/article/2012/01/02/pictures-report-idUSRTR2VTJ2
error http://www.reuters.com/article/2013/03/27/pictures-report-idUSRTXXYEH
error http://www.reuters.com/article/2012/06/14/idUS239816240020120614
error http://www.reuters.com/article/wiredStartUps/idUS97088668520091016
error http://www.reuters.com/article/2012/07/30/idUS372376970720120730
	success: 1595/1600
1600
error http://www.reuters.com/article/2012/10/30/idUS307499072820121030
error http://www.reuters.com/article/2010/12/21/idUS381407227820101221
error http://www.reuters.com/article/2012/08/20/pictures-report-idUSRTR36WMX
error http://www.reuters.com/article/2012/01/25/pictures-report-idUSRTR2WQY8
error http://www.reuters.com/article/2012/10/09/pictures-report-idUSRTR33ZDA
	success: 1595/1600
3200
error http://www.reuters.com/article/2011/01/22/idUS127285924720110122
error http://www.reuters.com/article/2010/10/17/idUS107226861620101017
	success: 1598/160

In [16]:
for idx in [37]:
    print('\n ------------------- %s --------------------\n'%idx)
    df1 = pd.read_csv('step1/%03d.csv'%idx, header=0)[['date','href','title']]
    df1 = df1[~df1.href.str.contains('\+')].reset_index(drop=True)
    i=0
    while i< df1.shape[0]:
        print(i)
        df1.loc[i: i+ batch_size-1, 'articles']= df1.loc[i: i+ batch_size-1, 'href'].parallel_apply(extract_story).values
        df_temp = df1.loc[i: i+ batch_size-1]
        print('\tsuccess: %s/%s'%(df_temp[~df_temp.articles.isnull()].shape[0], df_temp.shape[0]))
        i += batch_size
        if i% (batch_size * 10) ==0:
            df1.to_csv('step2/%03d.csv'%idx, index= False)
    df1.to_csv('step2/%03d.csv'%idx, index= False)


 ------------------- 37 --------------------

0
error http://www.reuters.com/article/yourView/idUSfbpryopcapgju31jqvuqtf2yzm8wxnid20100618
error http://www.reuters.com/article/yourView/idUSkrdamtallznszqfrmhwegpidfikn75qh20100702
error http://www.reuters.com/article/2010/12/13/idUS370745759120101213
error http://www.reuters.com/article/2010/12/16/idUS199014702220101216
error http://www.reuters.com/article/2012/12/14/idUS200017510320121214
error http://www.reuters.com/article/2012/07/02/pictures-report-idUSRTR33GKC
	success: 1594/1600
1600
error http://www.reuters.com/article/2012/11/30/idUS29367917620121130
error http://www.reuters.com/article/2011/03/04/pictures-report-idUSRTR2JFW3
error http://www.reuters.com/article/wiredGadgets/idUS92842890920100429
error http://www.reuters.com/article/yourView/idUS2xsfjidev8ixqbq0xwlvni0bcmq61tar20090821
error http://www.reuters.com/article/2010/12/30/idUS237540384220101230
	success: 1595/1600
3200
error http://www.reuters.com/article/2011/03/15/

In [3]:
pandarallel.initialize(nb_workers=8)
batch_size = 8*10

for idx in range(70,100):
    print('\n ------------------- %s --------------------\n'%idx)
    df1 = pd.read_csv('step1/%03d.csv'%idx, header=0)[['date','href','title']]
    df1 = df1[~df1.href.str.contains('\+')].reset_index(drop=True)
    i=0
    while i< df1.shape[0]:
        print(i)
        df1.loc[i: i+ batch_size-1, 'articles']= df1.loc[i: i+ batch_size-1, 'href'].parallel_apply(crawl_news_article).values
        df_temp = df1.loc[i: i+ batch_size-1]
        print('\tsuccess: %s/%s'%(df_temp[df_temp.articles.isnull()].shape[0], df_temp.shape[0]))
        i += batch_size
        if i% (batch_size * 10) ==0:
            df1.to_csv('step2/%03d.csv'%idx, index= False)
    df1.to_csv('step2/%03d.csv'%idx, index= False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ------------------- 70 --------------------

0
None	http://www.reuters.com/article/2011/12/09/idUS20734182620111209
None	http://www.reuters.com/article/2010/11/29/newzealand-press-idUSSGE6AS0FS20101129
None	http://www.reuters.com/article/2011/02/01/idUST18KCIHYO20110201
None	http://www.reuters.com/article/2012/10/18/sugar-ice-hours-idUSL5E8LIKKR20121018
None	http://www.reuters.com/article/2010/10/29/idUST2904K7N420101029
None	http://www.reuters.com/article/2011/02/16/israel-iran-whitehouse-idUSWEN794120110216
None	http://www.reuters.com/article/2011/01/12/idUS185393844820110112
None	http://www.reuters.com/article/2011/03/17/kumho-china-idUSTOE72G02M20110317
None	http://www.reuters.com/article/2011/09/29/diary-us-meetings-day-idUSL2970309620110929
None	http://www.reuters.com/article/2013/01/22/spain-debt-deguindos-idUSE8E8LN01620130122
None	http

KeyboardInterrupt: 

None	http://www.reuters.com/article/2011/08/11/estonia-incident-idUSLDE77A0XW20110811
None	http://www.reuters.com/article/2011/03/06/us-ivorycoast-border-idUSTRE7251B820110306
None	http://www.reuters.com/article/2011/01/10/michaelpage-idUSLDE70905N20110110
None	http://www.reuters.com/article/2011/03/14/us-bryanadams-idUSTRE72D7ZU20110314
None	http://www.reuters.com/article/2011/12/02/idUSWLB967020111202
None	http://www.reuters.com/article/2011/06/01/idUST18WGUFZO20110601
None	http://www.reuters.com/article/2010/09/16/idUSWLA311920100916
None	http://www.reuters.com/article/2011/03/25/eurozone-juncker-portugal-idUSBAT00611820110325
None	http://www.reuters.com/article/2011/11/04/idUST44CXVVJ820111104
404	https://web.archive.org/web/20180701203411/http://www.reuters.com/article/2012/11/15/gruma-idUSL1E8MFBR320121115
None	http://www.reuters.com/article/2011/09/22/idUST222JZBQ620110922
None	http://www.reuters.com/article/2012/03/28/germany-inflation-idUSL6E8ER9NI20120328
None	http://www.reut